In [30]:
import os
import sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from scipy.io.wavfile import read
from glob import glob
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image
from WaveNetClassifier import WaveNetClassifier # from https://github.com/mjpyeon/wavenet-classifier.git


In [31]:
def preprocess(data_path):
   try:
      array = read(data_path)[1]
      array = array.astype(np.float32)
      if array.ndim >= 2:
         array = np.mean(array, axis=1)
      if array.shape[0] > 96000:
         array = array[:96000]
         return array
      else:
         array = np.append(array, np.zeros(96000-array.shape[0]))
         pass
      return array
   except ValueError:
      pass



dog_state_list = ["hostile","relax","whining"]

X_train = []
y_train = []
for state in dog_state_list:
   state_data = glob(f"C:\\Users\\fisho\\FurEmotion-AI\\FurEmotion-AI\\origin_data\\dog\\{state}\\*.wav")
   for d_path in state_data:
      if not isinstance(preprocess(d_path), np.ndarray):
         continue
      X_train.append(preprocess(d_path))
      y_train.append(dog_state_list.index(state))
      
X_train = np.array(X_train)
y_train = np.array(y_train)

print(X_train.shape, y_train.shape)

valid_rate = 0.1

random_batch = np.random.choice(range(len(X_train)), int(len(X_train)*valid_rate), replace=False)

X_val = X_train[random_batch]
y_val = y_train[random_batch]

print(X_train.shape, y_train.shape)

X_train = np.delete(X_train, random_batch,axis=0)
y_train = np.delete(y_train, random_batch, axis=0)



(632, 96000) (632,)
(632, 96000) (632,)


In [32]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((569, 96000), (569,), (63, 96000), (63,))

In [33]:
epoch = 5
wnc = WaveNetClassifier((96000,), (3,), kernel_size = 2, dilation_depth = 9, n_filters = 40, task = 'classification')

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 original_input (InputLayer  [(None, 96000)]              0         []                            
 )                                                                                                
                                                                                                  
 reshaped_input (Reshape)    (None, 96000, 1)             0         ['original_input[0][0]']      
                                                                                                  
 dilated_conv_1 (Conv1D)     (None, 96000, 40)            120       ['reshaped_input[0][0]']      
                                                                                                  
 dilated_conv_2_tanh (Conv1  (None, 96000, 40)            3240      ['dilated_conv_1[0][0]'

In [35]:

wnc.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = epoch, batch_size = 32, optimizer='adam', save=True, save_dir='./')

AttributeError: 'WaveNetClassifier' object has no attribute 'task'